In [6]:
import keras
import keras.backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger
import os
import keras.activations
import time

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [8]:
batch_size = 64
epochs = 100
units = 64
experiments = 5
start = 0
# activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
activations = ['elu']
# activations = ['selu']

In [9]:
start_time = time.time()
counter = 0
total_items = len(activations) * experiments

for act in activations:
    print("Training for activation " + act + " with RMSProp optimizer")

    for i in range(experiments):

        K.clear_session()
        K.reset_uids()

        act_dict = {
          'sigmoid': Activation(keras.activations.sigmoid),
          'tanh': Activation(keras.activations.tanh),
          'relu': Activation(keras.activations.relu),
          'linear': Activation(keras.activations.linear),
          'elu': keras.layers.ELU(alpha=1.5),
          'softplus': Activation(keras.activations.softplus),
          'softsign': Activation(keras.activations.softsign),
          'hard_sigmoid': Activation(keras.activations.hard_sigmoid),
          'LeakyReLU': keras.layers.advanced_activations.LeakyReLU(alpha=1.5),
          'selu': Activation(keras.activations.selu),
          'ThresholdedReLU': keras.layers.advanced_activations.ThresholdedReLU(theta=0.7) 
        }

        model_name = 'normd_' + act + '_1.5_rmsp_' + str(i + start) + '_' + str(units)
        inputs = Input(shape=(784,))
        x = Dense(units)(inputs)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        x = Dense(units)(x)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        predictions = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)

        model.compile(loss='categorical_crossentropy',
                          optimizer=RMSprop(lr=0.001),
                          metrics=['accuracy'])

        csv_logger = CSVLogger('./training_logs/%s.csv' % (model_name), append=False)
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test), callbacks=[csv_logger])

        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        t = time.time()
        time_diff = t - start_time
        counter +=1
        rem_items = total_items - counter
        total_time = round((total_items / counter) * time_diff)
        rem_time = round(total_time - time_diff)
        m, s = divmod(rem_time, 60)
        h, m = divmod(m, 60)
        d, h = divmod(h, 24)
        print('Remaining time: %d days %d hours %02d minutes %02d seconds' % (d, h, m, s))

Training for activation elu with RMSProp optimizer
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.4268 - acc: 0.8736 - val_loss: 0.2440 - val_acc: 0.9272
Epoch 2/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.2867 - acc: 0.9154 - val_loss: 0.1887 - val_acc: 0.9438
Epoch 3/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.2336 - acc: 0.9305 - val_loss: 0.1587 - val_acc: 0.9516
Epoch 4/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.2064 - acc: 0.9398 - val_loss: 0.1406 - val_acc: 0.9568
Epoch 5/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.1848 - acc: 0.9452 - val_loss: 0.1253 - val_acc: 0.9617
Epoch 6/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.1726 - acc: 0.9471 - val_loss: 0.1116 - val_acc: 0.9656
Epoch 7/100
60000/60000 [==============================] - 2s 38us/step

60000/60000 [==============================] - 4s 73us/step - loss: 0.0732 - acc: 0.9786 - val_loss: 0.0874 - val_acc: 0.9788
Epoch 60/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0712 - acc: 0.9784 - val_loss: 0.0899 - val_acc: 0.9780
Epoch 61/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.0742 - acc: 0.9773 - val_loss: 0.0929 - val_acc: 0.9783
Epoch 62/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.0723 - acc: 0.9788 - val_loss: 0.0920 - val_acc: 0.9793
Epoch 63/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.0728 - acc: 0.9781 - val_loss: 0.0881 - val_acc: 0.9790
Epoch 64/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.0739 - acc: 0.9784 - val_loss: 0.0926 - val_acc: 0.9785
Epoch 65/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.0715 - acc: 0.9784 - val_loss: 0.0928 - val_acc: 0.9787
Epoch 66/100
60000/60000 [==============

Epoch 17/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.1068 - acc: 0.9676 - val_loss: 0.0872 - val_acc: 0.9748
Epoch 18/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.1027 - acc: 0.9686 - val_loss: 0.0841 - val_acc: 0.9745
Epoch 19/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.1004 - acc: 0.9693 - val_loss: 0.0842 - val_acc: 0.9757
Epoch 20/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.1014 - acc: 0.9695 - val_loss: 0.0868 - val_acc: 0.9757
Epoch 21/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.1007 - acc: 0.9706 - val_loss: 0.0784 - val_acc: 0.9778
Epoch 22/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.0942 - acc: 0.9717 - val_loss: 0.0769 - val_acc: 0.9770
Epoch 23/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.0951 - acc: 0.9713 - val_loss: 0.0816 - val_acc: 0.9767
Epoch 24/100
60000/60000 [=

Epoch 76/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.0647 - acc: 0.9807 - val_loss: 0.0913 - val_acc: 0.9797
Epoch 77/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0636 - acc: 0.9815 - val_loss: 0.0888 - val_acc: 0.9791
Epoch 78/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0627 - acc: 0.9817 - val_loss: 0.0848 - val_acc: 0.9794
Epoch 79/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.0626 - acc: 0.9818 - val_loss: 0.0880 - val_acc: 0.9788
Epoch 80/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.0634 - acc: 0.9814 - val_loss: 0.0870 - val_acc: 0.9796
Epoch 81/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0645 - acc: 0.9805 - val_loss: 0.0839 - val_acc: 0.9800
Epoch 82/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0654 - acc: 0.9812 - val_loss: 0.0867 - val_acc: 0.9789
Epoch 83/100
60000/60000 [=

Epoch 34/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.0853 - acc: 0.9749 - val_loss: 0.0792 - val_acc: 0.9797
Epoch 35/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.0829 - acc: 0.9753 - val_loss: 0.0823 - val_acc: 0.9781
Epoch 36/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.0856 - acc: 0.9750 - val_loss: 0.0774 - val_acc: 0.9793
Epoch 37/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.0802 - acc: 0.9760 - val_loss: 0.0812 - val_acc: 0.9786
Epoch 38/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.0821 - acc: 0.9748 - val_loss: 0.0803 - val_acc: 0.9784
Epoch 39/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.0807 - acc: 0.9754 - val_loss: 0.0815 - val_acc: 0.9789
Epoch 40/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.0801 - acc: 0.9759 - val_loss: 0.0835 - val_acc: 0.9784
Epoch 41/100
60000/60000 [=

Epoch 93/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0637 - acc: 0.9810 - val_loss: 0.0953 - val_acc: 0.9791
Epoch 94/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.0641 - acc: 0.9811 - val_loss: 0.0932 - val_acc: 0.9787
Epoch 95/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.0614 - acc: 0.9818 - val_loss: 0.0965 - val_acc: 0.9781
Epoch 96/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.0613 - acc: 0.9817 - val_loss: 0.0928 - val_acc: 0.9792
Epoch 97/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.0655 - acc: 0.9811 - val_loss: 0.0957 - val_acc: 0.9792
Epoch 98/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.0618 - acc: 0.9818 - val_loss: 0.0895 - val_acc: 0.9797
Epoch 99/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.0631 - acc: 0.9816 - val_loss: 0.0926 - val_acc: 0.9807
Epoch 100/100
60000/60000 [

Epoch 51/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0731 - acc: 0.9779 - val_loss: 0.0825 - val_acc: 0.9783
Epoch 52/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.0732 - acc: 0.9777 - val_loss: 0.0842 - val_acc: 0.9790
Epoch 53/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0755 - acc: 0.9778 - val_loss: 0.0822 - val_acc: 0.9790
Epoch 54/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0727 - acc: 0.9779 - val_loss: 0.0871 - val_acc: 0.9753
Epoch 55/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0721 - acc: 0.9782 - val_loss: 0.0832 - val_acc: 0.9778
Epoch 56/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0735 - acc: 0.9777 - val_loss: 0.0846 - val_acc: 0.9787
Epoch 57/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.0737 - acc: 0.9774 - val_loss: 0.0855 - val_acc: 0.9787
Epoch 58/100
60000/60000 [=

Epoch 9/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1409 - acc: 0.9566 - val_loss: 0.1035 - val_acc: 0.9691
Epoch 10/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1321 - acc: 0.9595 - val_loss: 0.0975 - val_acc: 0.9711
Epoch 11/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1273 - acc: 0.9610 - val_loss: 0.0977 - val_acc: 0.9710
Epoch 12/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1221 - acc: 0.9623 - val_loss: 0.0932 - val_acc: 0.9733
Epoch 13/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1199 - acc: 0.9628 - val_loss: 0.0973 - val_acc: 0.9712
Epoch 14/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1154 - acc: 0.9641 - val_loss: 0.0888 - val_acc: 0.9737
Epoch 15/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1103 - acc: 0.9668 - val_loss: 0.0846 - val_acc: 0.9758
Epoch 16/100
60000/60000 [==

Epoch 68/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.0660 - acc: 0.9794 - val_loss: 0.0960 - val_acc: 0.9771
Epoch 69/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.0647 - acc: 0.9802 - val_loss: 0.0918 - val_acc: 0.9792
Epoch 70/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.0667 - acc: 0.9795 - val_loss: 0.0901 - val_acc: 0.9794
Epoch 71/100
60000/60000 [==============================] - 4s 58us/step - loss: 0.0664 - acc: 0.9798 - val_loss: 0.0896 - val_acc: 0.9784
Epoch 72/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.0652 - acc: 0.9801 - val_loss: 0.0901 - val_acc: 0.9801
Epoch 73/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.0657 - acc: 0.9801 - val_loss: 0.0908 - val_acc: 0.9799
Epoch 74/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.0645 - acc: 0.9807 - val_loss: 0.0901 - val_acc: 0.9784
Epoch 75/100
60000/60000 [=